In [10]:
from dask.array import ones
from numpy import arange
import rioxarray
from xarray import DataArray, Dataset

same_shape_same_cell_size_data = ones(
    shape=(6, 6),
    dtype="uint8"
)

same_shape_same_cell_size_coords = {
    "northings": arange(5.5, 0, -1),
    "eastings": arange(0.5, 6, 1),
}

same_shape_same_cell_size_data_array = DataArray(
    data=same_shape_same_cell_size_data,
    coords=same_shape_same_cell_size_coords,
)

same_shape_same_cell_size_dataset = Dataset(
    data_vars={
        "same_shape_same_cell_size": same_shape_same_cell_size_data_array,
    },
    coords=same_shape_same_cell_size_coords,
)

(
    same_shape_same_cell_size_dataset
    .rio.set_spatial_dims(
        x_dim="eastings",
        y_dim="northings",
        inplace=True,
    )
    .rio.write_crs(
        27700, 
        inplace=True,
    )
)

same_shape_same_cell_size_dataset.rio.to_raster("same_shape_same_cell_size.tif")

In [15]:
data_path = "same_shape_same_cell_size.tif"
engine = None
decode_coords = "all"

In [16]:
from pathlib import Path

suffix = Path(data_path).suffixes[0]

In [17]:
from rasterio.drivers import raster_driver_extensions

if engine:
    _engine = engine
elif suffix == ".zarr":
    _engine = "zarr"
    _decode_coords = decode_coords
elif suffix[1:] in raster_driver_extensions().keys():
    _engine = "rasterio"
    _decode_coords = None

In [20]:
from xarray import open_dataset

dataset = open_dataset(
    data_path,
    engine=_engine,
    decode_coords=_decode_coords,
    mask_and_scale=False,
)

In [21]:
dataset

<xarray.Dataset>
Dimensions:      (band: 1, x: 6, y: 6)
Coordinates:
  * band         (band) int64 1
  * x            (x) float64 0.5 1.5 2.5 3.5 4.5 5.5
  * y            (y) float64 5.5 4.5 3.5 2.5 1.5 0.5
    spatial_ref  int64 ...
Data variables:
    band_data    (band, y, x) uint8 ...